#### Model Training
#### Best Model
#### tuning the best model

In [88]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
#models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
#Grid Search
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold
#Pipeline
from sklearn.pipeline import Pipeline
#metrics
from sklearn.metrics import accuracy_score

In [89]:
models={
    "LogisticRegression":LogisticRegression(),
    "DecisionTreeClassifier":DecisionTreeClassifier(),
    "KNeighborsClassifier":KNeighborsClassifier(),
    "SVC":SVC(),
    "GaussianNB":GaussianNB(),
    "MultinomialNB":MultinomialNB(),
    "AdaBoostClassifier":AdaBoostClassifier(),
    "GradientBoostingClassifier":GradientBoostingClassifier(),
    "RandomForestClassifier":RandomForestClassifier(),
    "XGBClassifier":XGBClassifier()
}

In [90]:
def model_training(models,x,y,flag):
    xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=0)
    acc_res={}
    acc_list=[]
    for name,model in models.items():
        try:
            if(flag==1):
                model_pipeline=Pipeline([("StandardScaler",StandardScaler()),('model',model)])
            elif(flag==2):
                 model_pipeline=Pipeline([("MinMaxScaler",MinMaxScaler()),('model',model)])
            else:
                model_pipeline=Pipeline([('model',model)])
                
            model_fit=model_pipeline.fit(xtrain,ytrain)
            ypred=model_fit.predict(xtest)
            acc=accuracy_score(ytest,ypred)
            acc_list.append(acc)
            acc_res[name]=acc
        except Exception as e:
            print("Opps Something whent wrong!",e)
    print(pd.DataFrame({"Model":models.keys(),"Accuracy":acc_list}))
    return acc_res
                    

In [91]:
def bestModel(model_res):
    high=0
    for name,acc in model_res.items():
        if acc>high:
            high=acc
            model_name=name
    print("Best Model is ",model_name," with accuracy ",high)

In [92]:
def bestParameters(model,params,xtrain,ytrain):
    cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=3)
    grid_cv=GridSearchCV(estimator=model,param_grid=params,cv=cv,scoring="accuracy")
    res=grid_cv.fit(xtrain,ytrain)
    print("Best Parameters :",res.best_params_)
    print("Best Accuracy :",res.best_score_)

In [93]:
df=pd.read_csv('../Data/sonar.all-data.csv',header=None)

In [94]:
df[60]=df[60].map({'M':0,"R":1})

In [95]:
x=df.drop(columns=[60])
y=df[60]
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=0)

In [96]:
model_resullt=model_training(models,x,y,1)

Opps Something whent wrong! Negative values in data passed to MultinomialNB (input X)


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[12:19:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ValueError: arrays must all be same length

In [ ]:
bestModel(model_resullt)

In [ ]:
model=GradientBoostingClassifier()
grid=dict(n_estimators=[10,50,100,500])
bestParameters(model,grid,x,y)